---
title: "Plotly Graph Objects"
---


## Motivation

Using `plotly.express` is generally easy and straightforward, but is limited
to a large set of useful but standard visualizations. These might not be the
best we can do for our data and story. `plotly` provides much more **granular**
control over the visualizations via the `plotly.graph_objects` module. This
module is very close to being a translation of the underlying _JavaScript_
library that is running this show, and allows you to control almost all aspects
of your visualization, along with adding multiple numerous interactive features.

::: {.callout-tip}
### Why use `plotly.graph_objects`?
If you want graphs that are more customized, or that need more control
structures for interactivity, you switch to `plotly.graph_objects`. There is no
incompatibility between `plotly.express` and `plotly.graph_objects`, since the
end product of `plotly.express` is a `plotly.graph_objects.Figure` object.


 For example, sliders, maps, and drop-down menus are easy individually with
 `plotly.express`, however *combining* them on the same visualization is non-trivial and needs `plotly.graph_objects`
 is
 non-trivial.
:::

 [![](img/2023-03-12-14-44-39.png){width=600}](img/2023-03-12-14-44-39.png)

::: {.callout-warning}
Working on recreating this plot. Link below doesn't work
:::

* ~~[Click here for interactive version of the map example](https://jfh.georgetown.domains/motivating-example.html)~~

## Structure of a graph object

* Plotly figures (`plotly.graph_object.Figure` objects) are represented as hierarchical trees, with `graph_object` being the root note, and child-nodes called `attributes`.
* Graph_objects have three top-level attributes: `data`, `layout`, and `frames` (frames are only needed for animated plots)
  * Notice the connection of these attributes to pure `JavaScript`.
* **Data:** This is a list of dictionaries referred to as "traces"
  - The `trace` represents a set of related graphical marks in a figure.
  - Each trace must have a `type` attribute which defines the other allowable attributes.
  - Each trace has one of more than 40 possible types (see below for a list organized by subplot type, including e.g. [`scatter`](https://plotly.com/python/line-and-scatter/), [`bar`](https://plotly.com/python/bar-charts/), [`pie`](https://plotly.com/python/pie-charts/), [`surface`](https://plotly.com/python/3d-surface-plots/), [`choropleth`](https://plotly.com/python/choropleth-maps/) etc)
* **Layout**: Controls various structural and stylistic components (e.g. title, font, size, etc)

::: {.aside}
Sources: [figure structure](https://plotly.com/python/figure-structure/) [Graph objects](https://plotly.com/python/graph-objects/)
:::

::: {.callout-info}
This structure of the `plotly.graph_objects.Figure` are directly parallel to the
structure of the `plotly.js` figure Objects
  - The data object is a list of JSON objects, each of which defines a single
  trace
  - The layout object is a JSON object that defines the overall layout of the figure.
:::

## Graph_object Structure summary

![](img/2023-03-12-17-03-01.png)

## Various available traces

* **2D Cartesian trace types, and Subplots**.
    [![](img/2023-03-12-15-34-01.png){width=720}](img/2023-03-12-15-34-01.png)
* **Geo-spatial trace Types**.
    [![](img/2023-03-12-15-34-35.png){width=720}](img/2023-03-12-15-34-35.png)

<sup>For more [click here](https://plotly.com/python/figure-structure/)!<sup>


## Looking under the hood

:::: {.columns}
::: {.column width="50%"}
* Viewing the underlying data structure for any plotly.graph_objects.Figure object can be done via `print(fig)` or  `fig.show("json")`.
* Figures also support `fig.to_dict()` and `fig.to_json()` methods.
:::
::: {.column width="50%"}
![](img/2023-03-10-12-08-27.png){width=400}
:::
::::


In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(x=["a", "b", "c"], y=[1, 3, 2], title="sample figure")
print(fig)
# fig.show()

## Graph_objects: Hello world

* When using graph objects (without animation), you typically do the following
  * `(1)` Initialize the figure
  * `(2)` Add one or more traces
  * `(3)` customize the layout


In [ ]:
import plotly.graph_objects as go

# DATAFRAME
df = px.data.gapminder()

# INITIALIZE GRAPH OBJECT
fig = go.Figure()

# ADD TRACES FOR THE DATA-FRAME
fig.add_trace(  # Add A trace to the figure
    go.Scatter(  # Specify the type of the trace
        x=df["gdpPercap"],  # Data-x
        y=df["lifeExp"],  # Data-y
        mode="markers",
        # note the re-normalization of population to map to width to units of "pixels"
        marker=dict(
            size=50 * (df["pop"] / max(df["pop"])) ** 0.5,
            color=df["pop"],
            showscale=True,
            colorscale="Viridis",
            symbol="circle",
        ),
        opacity=1.0,
    )
)

# SET THEME, AXIS LABELS, AND LOG SCALE
fig.update_layout(
    template="plotly_white",
    xaxis_title="National GDP (per capita)",
    yaxis_title="Life expectancy (years)",
    title="Country comparison: color & size = population",
    height=400,
    width=800,
)
fig.update_xaxes(type="log")

fig.show()

## Basic charts with graph_objects

![](img/2023-03-12-15-27-15.png){width=650}
<!-- [![](img/2023-03-12-15-27-15.png){width=600}](img/2023-03-12-15-27-15.png) -->

<!-- <sup> [Source](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) <sup>  -->

## Additional charts with Graph objects

![](img/2023-03-12-15-27-51.png){width=775}
<!-- [![](img/2023-03-12-15-27-51.png){width=600}](img/2023-03-12-15-27-51.png) -->

<!-- <sup> [Source](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) <sup>  -->

## Subplots

[**A coding note:** Notice that there are semi-colons (`;`) at the end of most
lines of code. This prevents the figure from printing out output of that line of
code or that function, especially within a
Quarto document. This is useful for **not** printing out intermediate steps and
lets only the final figure be printed.]{.aside}

With `plotly.graph_objects` we can finally combine figures in a subplot! [source](https://plotly.com/python/subplots/)


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2); # <1>

fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
    row=1, col=1 # <2>
);

fig.add_trace(
    go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
       row=1, col=2
);

fig.update_layout(height=500, width=700, title_text="Side By Side Subplots");
fig.show()

1. Specifying the arrangement of subplots
2. Specifying the position of the first graph in the pre-specified subplot
   arrangement

## Multiple Subplots

Here we show a 2 x 2 subplot grid with each subplot populated with a single scatter trace.


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2, start_cell="bottom-left");

fig.add_trace(
  go.Scatter(x=[1, 2, 3], y=[4, 5, 6], name = "Plot 1"),
  row=1, col=1
);

fig.add_trace(
  go.Scatter(x=[20, 30, 40], y=[50, 60, 70], name = "Plot 2"),
  row=1, col=2,
);

fig.add_trace(
  go.Scatter(x=[300, 400, 500], y=[600, 700, 800], name = "Plot 3"),
  row=2, col=1
);

fig.add_trace(
  go.Scatter(x=[4000, 5000, 6000], y=[7000, 8000, 9000], name = "Plot 4"),
              row=2, col=2,
);
fig.show()

## Demonstration of more capabilities

### Import


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

### Data

* We can generate a graph similar to the motivating example, using the gap-minder dataset.
* Let's briefly explore this.


In [ ]:
# DATAFRAME
df = px.data.gapminder()
df = df.drop(['iso_num'], axis=1)   # DROP COLUMN
print("Shape =", df.shape)
print(df)

### Choropleth

* We will cover `Choropleths` in more detail during the `Geo-spatial module`. However, for now, all you need to know is that a choropleth is a type of map that uses colors or shading to represent different values or levels of a particular data variable across a geographic area, such as a country, state, or city.

* The choropleth map divides the area into regions or polygons, usually based on administrative boundaries, and then assigns a color or shade to each region based on the value of the data variable being represented. For example, if the data variable is population density, then regions with higher population density would be shaded darker than regions with lower population density.

* Plotly can internally create the map using location tags for countries, such as `USA`.


In [ ]:
# ISOLATE ONE YEAR OF DATA FOR PLOTTING
df = df.query("year==2007")

# INITIALIZE GRAPH OBJECT
fig = go.Figure();

# ADD A CHOROPLETH TRACES FOR THE DATA-FRAME
fig.add_trace(  # Add a trace to the figure
    go.Choropleth(  # Specify the type of the trace
        uid="full-set",  # uid=unique id (Assign an ID to the trace)
        locations=df["iso_alpha"],  # Supply location information tag for mapping
        z=df["lifeExp"],  # Data to be color-coded on graph
        colorbar_title="Life expectancy",  # Title for color-bar
        visible=True,  # <1> Specify whether or not to make data-visible when rendered
    )
);

# SHOW
fig.show()

1. The final input argument `visible` may seem silly, obviously we want to see the data! However, this becomes quite important when you want to *conditionally* update different traces.

### Drop-down menus

* `Dropdown menus` are very important for interactivity, since they let the user jump between data-sets, graph-types, or plotting styles.
* Drop-down menus are controlled in `plotly graph objects` using four methods, which tell `plotly.js` how to modify the chart.
  - `restyle`:  modify data or data attributes
  - `relayout`: modify layout attributes
  - `update`: modify data **and** layout attributes
  - `animate`: start or pause an [animation](https://plotly.com/python/#animations)

* For more see: [https://plotly.com/python/dropdowns/](https://plotly.com/python/dropdowns/)

#### Dropdown syntax

In Plotly, adding drop-down menus typically follows the following code-format.

```python
# ADD DROPDOWNS
fig.update_layout(
    updatemenus=[
        # BUTTON-1
        dict(
            # BUTTON-1 OPTIONS
            buttons=list([
                dict(
                    ),
                dict(
                    ),
                        ]),
            # BUTTON-1 META-DATA
            direction="left",
            ),

        # BUTTON-2
        dict(
            # BUTTON-2 OPTIONS
            buttons=list([
                dict(
                    ),
                dict(
                    )
                        ]),
            # BUTTON-2 META-DATA
            direction="down",
            ),
    ]
)
```

#### Example 1

* Changing something that doesn't require switching the data being plotted is relatively straight-forward (e.g. the type of plot or color)
* Make sure to take note of the various comments which high-light relevant information!


In [ ]:
# INITIALIZE GRAPH OBJECT
fig = go.Figure();

# ADD TRACES FOR THE DATA-FRAME
fig.add_trace(  # Add A trace to the figure
    go.Scatter(  # Specify the type of the trace
        x=df["gdpPercap"],  # Data-x
        y=df["lifeExp"],  # Data-y
        mode="markers",
        # note the re-normalization of population to map to width to units of "pixels"
        marker=dict(
            size=50 * (df["pop"] / max(df["pop"])) ** 0.5,
            color=df["pop"],
            showscale=True,
            colorscale="Viridis",
            symbol="circle",
        ),
        opacity=1.0,
        visible=True,  # Specify whether or not to make data-visible when rendered
    )
);

# SET THEME, AXIS LABELS, AND LOG SCALE
fig.update_layout(
    template="plotly_white",
    xaxis_title="National GDP (per capita)",
    yaxis_title="Life expectancy (years)",
    title="Country comparison: color & size = population"
);
fig.update_xaxes(type="log");


# VARIABLES FOR BUTTON LOCATION
# (SET THESE BY TRIAL AND ERROR)
button_height = 0.15
x1_loc = 0.65
y1_loc = 1.2
x2_loc = x1_loc
y2_loc = y1_loc + 2 * button_height
x3_loc = x1_loc
y3_loc = y1_loc + 4 * button_height

# DROPDOWN MENUS
fig.update_layout(
    # DEFINE A LIST OF THE VARIOUS BUTTONS (STORED AS DICTIONARIES)
    updatemenus=[
        # BUTTON-1: OPACITY
        dict(
            # NOTICE THAT THE OPTION ARE A LIST OF DICTIONARIES
            # IMPORTANT: args specifies the key-value pairing for what to change
            buttons=[
                dict(
                    label="1.00",               # LABEL SHOWN TO USER
                    method="restyle",           # MODIFICATION TYPE (SEE ABOVE)
                    args=["opacity", 1.00],     # KEY-VALUE FOR WHAT TO CHANGE AND HOW
                ),
                dict(
                    label="0.75",
                    method="restyle",
                    args=["opacity", 0.75],
                ),
                dict(
                    label="0.50",
                    method="restyle",
                    args=["opacity", 0.5],
                ),
                dict(
                    label="0.25",
                    method="restyle",
                    args=["opacity", 0.25],
                ),
            ],
            # PLACEMENT AND META DATA FOR THE BUTTON
            direction="right",
            showactive=True,  # HIGHLIGHTS ACTIVE DROPDOWN ITEM OR ACTIVE BUTTON IF TRUE
            pad={"r": 10, "t": 10},  # PADDING
            x=x1_loc,  # POSITION
            y=y1_loc,
            xanchor="left",  # ANCHOR POINT
            yanchor="top",
        ),
        # BUTTON-2: COLOR SCALE
        dict(
            buttons=list(
                [
                    dict(
                        args=["marker.colorscale", "Viridis"],
                        label="Viridis",
                        method="restyle",
                    ),
                    dict(
                        args=["marker.colorscale", "Cividis"],
                        label="Cividis",
                        method="restyle",
                    ),
                    # dict(
                    #     args=["marker.colorscale", "Blues"],
                    #     label="Blues",
                    #     method="restyle",
                    # ),
                    dict(
                        args=["marker.colorscale", "Greens"],
                        label="Greens",
                        method="restyle",
                    ),
                ]
            ),
            # PLACEMENT AND META DATA FOR THE BUTTON
            direction="right",
            showactive=True,  # HIGHLIGHTS ACTIVE DROPDOWN ITEM OR ACTIVE BUTTON IF TRUE
            pad={"r": 10, "t": 10},  # PADDING
            x=x2_loc,  # POSITION
            y=y2_loc,
            xanchor="left",  # ANCHOR POINT
            yanchor="top",
        ),
        # BUTTON-3: MARKER SYMBOL
        dict(
            # NOTICE THAT THE OPTION ARE A LIST OF DICTIONARIES
            # args specifices the key-value pairing for what to change
            buttons=list(
                [
                    dict(
                        label="Circle",
                        method="restyle",    # changing style
                        args=["marker.symbol", "circle"],
                    ),
                    dict(
                        label="Square",
                        method="restyle",
                        args=["marker.symbol", "square"],
                    ),
                    dict(
                        label="Diamond",
                        method="restyle",
                        args=["marker.symbol", "diamond"],
                    ),
                ]
            ),
            # PLACEMENT AND META DATA FOR THE BUTTONS
            direction="right",
            showactive=True,  # HIGHLIGHTS ACTIVE DROPDOWN ITEM OR ACTIVE BUTTON IF TRUE
            pad={"r": 10, "t": 10},  # PADDING
            x=x3_loc,  # POSITION
            y=y3_loc,
            xanchor="left",  # ANCHOR POINT
            yanchor="top",
        ),
    ]
);

# ADD ANNOTATION TO LABEL BUTTONS
fig.update_layout(
    annotations=[
        dict(
            text="Opacity",
            x=x1_loc+0.08,
            xref="paper",
            y=y1_loc + 0.06,
            yref="paper",
            showarrow=False,
        ),
        dict(
            text="Color scale",
            x=x2_loc+ 0.11 ,
            xref="paper",
            y=y2_loc + 0.06,
            yref="paper",
            showarrow=False,
        ),
        dict(
            text="Shape",
            x=x3_loc + 0.07,
            xref="paper",
            y=y3_loc + 0.06,
            yref="paper",
            showarrow=False,
        ),
    ]
);

fig.show()

#### Example 2:

You can control which data is shown by controlling which traces are visible. This is done with a list of boolean values for the visibility of each trace `[False, True]` (see below).

The "update" method should be used when modifying the data and layout sections of the graph.

This example demonstrates how to update which traces are displayed.


In [ ]:
# INITIALIZE GRAPH OBJECT
fig = go.Figure()

# TRACE-1: gdpPercap vs lifeExp (area=pop)
fig.add_trace(  # Add A trace to the figure
    go.Scatter(  # Specify the type of the trace
        x=df["gdpPercap"],  # Data-x
        y=df["lifeExp"],  # Data-y
        mode="markers",
        # note the re-normalization of population to map to width to units of "pixels"
        marker=dict(
            color=df["lifeExp"],
            showscale=True,
            colorscale="Viridis",
            symbol="circle",
        ),
        opacity=1.0,
        visible=True,  # Specify whether or not to make data-visible when rendered
    )
);

# TRACE-2: gdpPercap vs pop
fig.add_trace(  # Add A trace to the figure
    go.Scatter(  # Specify the type of the trace
        x=df["gdpPercap"],  # Data-x
        y=df["pop"],  # Data-y
        mode="markers",
        # note the re-normalization of population to map to width to units of "pixels"
        marker=dict(
            color=df["pop"],
            showscale=True,
            colorscale="Viridis",
            symbol="circle",
        ),
        opacity=1.0,
        visible=False,  # Specify whether or not to make data-visible when rendered
    )
);

# SET THEME, AXIS LABELS, AND LOG SCALE
fig.update_layout(
    template="plotly_white",
    xaxis_title="National GDP (per capita)",
    title="Life expectancy (years)",
);
fig.update_xaxes(type="log");


# VARIABLES FOR BUTTON LOCATION
# (SET THESE BY TRIAL AND ERROR)
button_height = 0.15
x1_loc = 0.00
y1_loc = 1.15

# DROPDOWN MENUS
fig.update_layout(
    # DEFINE A LIST OF THE VARIOUS BUTTONS (STORED AS DICTIONARIES)
    updatemenus=[
        # BUTTON-1: OPACITY
        dict(
            # NOTICE THAT THE OPTION ARE A LIST OF DICTIONARIES
            # IMPORTANT: args specifies the key-value pairing for what to change
            buttons=[
                dict(
                    label="Life expectancy",            # LABEL SHOWN TO USER
                    method="update",                    # MODIFICATION TYPE (SEE ABOVE)
                     args=[{"visible": [True, False]},  # BOOLEAN VALUES FOR EACH TRACE
                           {"title": "Life expectancy (years)"}]
                     ),
                dict(
                    label="Population",               # LABEL SHOWN TO USER
                    method="update",           # MODIFICATION TYPE (SEE ABOVE)
                     args=[{"visible": [False, True]},
                           {"title": "Population"}]
                     ),
            ],
            # PLACEMENT AND META DATA FOR THE BUTTON
            direction="down",
            showactive=True,  # HIGHLIGHTS ACTIVE DROPDOWN ITEM OR ACTIVE BUTTON IF TRUE
            pad={"r": 10, "t": 10},  # PADDING
            x=x1_loc,  # POSITION
            y=y1_loc,
            xanchor="left",  # ANCHOR POINT
            yanchor="top",
        )
    ]
);


fig.show()

### Sliders

#### Example 1

Sliders can be used in Plotly to change the data displayed (using multiple traces) or style of a plot.


In [ ]:
#source: https://plotly.com/python/sliders/
import plotly.graph_objects as go
import numpy as np

# initialize figure
fig = go.Figure();

# Add traces, one for each slider step
for step in np.arange(0, 5, 0.1):
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            name="𝜈 = " + str(step),
            x=np.arange(0, 10, 0.01),
            y=np.sin(step * np.arange(0, 10, 0.01))));

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
);

fig.update_layout(template="plotly_white");


fig.show()

#### Example 2

Not surprisingly, sliders are trivial with Plotly express.


In [ ]:
import plotly.express as px

df = px.data.gapminder()
fig = px.scatter(df, x="gdpPercap", y="lifeExp",
                 animation_frame="year", animation_group="country", # <1>
                 size="pop", color="continent", hover_name="country",
                 log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90]);

# SET THEME
fig.update_layout(template="plotly_white");

fig["layout"].pop("updatemenus");  # optional, drop animation buttons
fig.show()

1. These two arguments make the slider and animation work.

You may remember this graph from the video we showed earlier in the semester ... pretty neat!

[Hans Rosling's 200 Countries, 200 Years, 4 Minutes](https://youtu.be/jbkSRLYSojo)

## Resources

* The documentation on the Plotly website is very good: [https://plotly.com/python/](https://plotly.com/python/)
* This provides a massive collection of examples, for both `plotly.express` and `plotly.graph_objects`, which can get you started on almost any task.

![](img/2023-03-12-15-10-34.png){width=750}